Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# AutoML 11: Sample weight

In this example we use the scikit learn's [digit dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) to showcase how you can use sample weight with the AutoML Classifier.
Sample weight is used where some sample values are more important than others.

Make sure you have executed the [00.configuration](00.configuration.ipynb) before running this notebook.

In this notebook you would see
1. How to specifying sample_weight
2. The difference that it makes to test results



## Create Experiment

As part of the setup you have already created a <b>Workspace</b>. For AutoML you would need to create an <b>Experiment</b>. An <b>Experiment</b> is a named object in a <b>Workspace</b>, which is used to run experiments.

In [ ]:
import logging
import os
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'non_sample_weight_experiment'
sample_weight_experiment_name = 'sample_weight_experiment'

# project folder
project_folder = './sample_projects/automl-local-classification'

experiment=Experiment(ws, experiment_name)
sample_weight_experiment=Experiment(ws, sample_weight_experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

## Diagnostics

Opt-in diagnostics for better experience, quality, and security of future releases

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

## Instantiate Auto ML Config

Instantiate two AutoMLConfig Objects. One will be used with sample_weight and one without.

In [ ]:
digits = datasets.load_digits()
X_digits = digits.data[100:,:]
y_digits = digits.target[100:]

# The example makes the sample weight 0.9 for the digit 4 and 0.1 for all other digits.
# This makes the model more likely to classify as 4 if the image it not clear.
sample_weight = np.array([(0.9 if x == 4 else 0.01) for x in y_digits])

automl_classifier = AutoMLConfig(task = 'classification',
                                 debug_log = 'automl_errors.log',
                                 primary_metric = 'AUC_weighted',
                                 max_time_sec = 3600,
                                 iterations = 10,
                                 n_cross_validations = 2,
                                 verbosity = logging.INFO,
                                 X = X_digits, 
                                 y = y_digits,
                                 path=project_folder)

automl_sample_weight = AutoMLConfig(task = 'classification',
                                    debug_log = 'automl_errors.log',
                                    primary_metric = 'AUC_weighted',
                                    max_time_sec = 3600,
                                    iterations = 10,
                                    n_cross_validations = 2,
                                    verbosity = logging.INFO,
                                    X = X_digits, 
                                    y = y_digits,
                                    sample_weight = sample_weight,
                                    path=project_folder)

## Training the Models

Call the submit method on the experiment and pass the configuration. For Local runs the execution is synchronous. Depending on the data and number of iterations this can run for while.
You will see the currently running iterations printing to the console.

In [ ]:
local_run = experiment.submit(automl_classifier, show_output=True)
sample_weight_run = sample_weight_experiment.submit(automl_sample_weight, show_output=True)

best_run, fitted_model = local_run.get_output()
best_run_sample_weight, fitted_model_sample_weight = sample_weight_run.get_output()

### Testing the Fitted Models

#### Load Test Data

In [ ]:
digits = datasets.load_digits()
X_digits = digits.data[:100, :]
y_digits = digits.target[:100]
images = digits.images[:100]

#### Compare the pipelines
The prediction from the sample weight model is more likely to correctly predict 4's.  However, it is also more likely to predict 4 for some images that are not labelled as 4.

In [ ]:
#Randomly select digits and test
for index in range(0,len(y_digits)):
    predicted = fitted_model.predict(X_digits[index:index + 1])[0]
    predicted_sample_weight = fitted_model_sample_weight.predict(X_digits[index:index + 1])[0]
    label = y_digits[index]
    if predicted == 4 or predicted_sample_weight == 4 or label == 4:
        title = "Label value = %d  Predicted value = %d Prediced with sample weight = %d" % ( label,predicted,predicted_sample_weight)
        fig = plt.figure(1, figsize=(3,3))
        ax1 = fig.add_axes((0,0,.8,.8))
        ax1.set_title(title)
        plt.imshow(images[index], cmap=plt.cm.gray_r, interpolation='nearest')
        plt.show()